In [1]:
import os
import subprocess
import sys
import datetime
import multiprocessing
import traceback
from dateutil.relativedelta import relativedelta

#source = "/gpfs/glad3/HLSDIST/testing/TSmodels/230612"#TSmodels/1_shutdown5obs3months"#"/gpfs/glad3/HLSDIST/VFModel/Drone/LP-DAAC"#"/gpfs/glad3/HLSDIST/LP-DAAC"
source = "/gpfs/glad3/HLSDIST/Validation"
alertsource = "/gpfs/glad3/HLSDIST/testing/v1sample/DIST-ALERT_v1"#LP-DAAC/DIST-ALERT_v1"
ANNname = "v1sample"
outfolder = "../mapLabels"+ANNname#_newVF"
#os.makedirs(outfolder)
ANNlayers = ['VEG-DIST-STATUS','VEG-ANOM-MAX','VEG-IND-MAX','VEG-DIST-CONF','VEG-DIST-DATE','VEG-DIST-DUR','VEG-DIST-COUNT','GEN-DIST-STATUS','GEN-ANOM-MAX','GEN-DIST-CONF','GEN-DIST-DATE','GEN-DIST-DUR','GEN-DIST-COUNT']
ALERTlayers = ['VEG-DIST-STATUS','VEG-ANOM','VEG-IND','VEG-HIST','VEG-ANOM-MAX','VEG-DIST-CONF','VEG-DIST-DATE','VEG-DIST-COUNT','VEG-DIST-DUR','VEG-LAST-DATE','GEN-DIST-STATUS','GEN-ANOM','GEN-ANOM-MAX','GEN-DIST-CONF','GEN-DIST-DATE','GEN-DIST-COUNT','GEN-DIST-DUR','GEN-LAST-DATE']
startdate = datetime.datetime.strptime("2020-12-31","%Y-%m-%d")

sampleDict = {}
sampleFull = {}
with open ('sampledpixels1214.csv','r') as dat:
  lines = dat.readlines()[1:]
  for l in lines:
    (ID,Stratum,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS) = l.strip().split(',')
    sampleDict[ID] = ','.join([Long,Lat,zone,centxUTM,centyUTM])
    sampleFull[ID] = l.strip()

In [4]:
def extractALERTsingle(line):#(ID,Long,Lat,MGRS):
  (ID,Stratum,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS) = str(line).split(',')
  #outfolder = "sample_v1"#""testPixels2023"
  #ANNname = ""
  print(ID)
  Long = str(Long)
  Lat = str(Lat)
  with open(outfolder+'/'+ID+"_DIST-ALERT_"+ANNname+".csv",'w') as alertcsv:
    alertcsv.write("granuleID,SensingTime,ProductionTime,VEG-DIST-STATUS,VEG-ANOM,VEG-IND,VEG-HIST,VEG-ANOM-MAX,VEG-DIST-CONF,VEG-DIST-DATE,VEG-DIST-COUNT,VEG-DIST-DUR,VEG-LAST-DATE,GEN-DIST-STATUS,GEN-ANOM,GEN-ANOM-MAX,GEN-DIST-CONF,GEN-DIST-DATE,GEN-DIST-COUNT,GEN-DIST-DUR,GEN-LAST-DATE\n")
    tilepath = MGRS[0:2]+'/'+MGRS[2:3]+'/'+MGRS[3:4]+'/'+MGRS[4:5]
    response = subprocess.run(["ls "+alertsource+"/202[1-2]/"+tilepath+"/DIST-ALERT*/OPERA*VEG-DIST-STATUS.tif 2>/dev/null"], capture_output=True,shell=True)
    Tgranlist = str(response.stdout.decode())
    Tgranlist = Tgranlist.strip().split('\n')
    granlist = []
    for filepath in Tgranlist:
      folders = filepath.split('/')
      gid = folders[-1]
      (gid,temp)= gid.split('.')
      granlist.append(gid+","+filepath)
      #print(granlist)
    for line in granlist:
      (gID,granpath) = line.split(',')
      #(p1,p2) = granpath.split('//')
      #granpath = p1+'/DIST-ALERT/'+p2
      folders = granpath.split('/')
      (sOPERA,sL3,sDIST,sTtile,ssensingTime,sProdTime,ssatellite,sres,sDISTversion,tlayer)=gID.split('_')
      ODIST_ID = '_'.join([sOPERA,sL3,sDIST,sTtile,ssensingTime,sProdTime,ssatellite,sres,sDISTversion])
      granpath = '/'.join(folders[0:-1])+"/"+ODIST_ID
      stime = datetime.datetime.strptime(ssensingTime,"%Y%m%dT%H%M%SZ")
      ptime = datetime.datetime.strptime(sProdTime,"%Y%m%dT%H%M%SZ")
      alertcsv.write(','.join([gID,stime.strftime("%Y%m%d"),str(ptime)]))
      for layer in ALERTlayers:
        #print("gdallocationinfo -wgs84 -valonly " + granpath+'_'+layer+".tif "+Long+" "+Lat)
        response = subprocess.run(["gdallocationinfo -wgs84 -valonly " + granpath+'_'+layer+".tif "+Long+" "+Lat], capture_output=True,shell=True)
        #print("gdallocationinfo -wgs84 -valonly " + granpath+'_'+layer+".tif "+Long+" "+Lat)
        value = str(response.stdout.decode().strip()).split('\n')[0]
        #print(value)
        if layer == "VEG-DIST-DATE" or layer == "GEN-DIST-DATE":
          if int(value) > 0:
            date = startdate+datetime.timedelta(days = int(value))
            value = date.strftime("%Y%m%d")
          else:
            value = "NA"
        alertcsv.write(','+str(value))
        #print(','+str(value),end='')
      alertcsv.write("\n")
      #print()
        
def processQueue(procID,queue,sampletable):
  Nprocess = 0
  while not queue.empty():
    ID = queue.get().strip()
    try:
      extractALERTsingle(sampletable[ID])
      #extractVFsingleBaselineTight(sampletable[ID])
      #extractVFsingleBaseline(sampletable[ID])
      Nprocess +=1
    except:
      traceback.print_exc()
  return Nprocess

def extractALERT():
  pixelqueue = multiprocessing.Queue()
  sampletable = {}
  for line in lines:
    (ID,Stratum,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS) = line.strip().split(',')
    sampletable[ID] = line.strip()
  for ID in range(1,301):
    pixelqueue.put(str(ID))
  processes = []
  for procID in range(1,61):
    proc = multiprocessing.Process(target=processQueue,args=(str(procID),pixelqueue,sampletable))
    processes.append(proc)
    proc.start()
  for p in processes:
    p.join()
  pixelqueue.close()
  pixelqueue.join_thread()  
  print("ALERT done")

In [5]:
extractALERT()

1
2
3
4
5
6
7
8
9
10
15
16
17
11
12
18
19
13
20
14
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
def makeMapTimeSeriesDaily(layer,nodatavalue):
  nodatavalue = str(nodatavalue)
  layers = ['granuleID','sensingTime','ProductionTime','VEG-DIST-STATUS','VEG-ANOM','VEG-IND','VEG-HIST','VEG-ANOM-MAX','VEG-DIST-CONF','VEG-DIST-DATE','VEG-DIST-COUNT','VEG-DIST-DUR','VEG-LAST-DATE','GEN-DIST-STATUS','GEN-ANOM','GEN-ANOM-MAX','GEN-DIST-CONF','GEN-DIST-DATE','GEN-DIST-COUNT','GEN-DIST-DUR','GEN-LAST-DATE']
  #layers = ['granuleID','sensingTime','ProductionTime','VEG-DIST-STATUS','VEG-ANOM','VEG-IND','VEG-DIST-CONF','VEG-DIST-DATE','VEG-DIST-DUR','GEN-DIST-STATUS','GEN-ANOM','GEN-DIST-CONF','GEN-DIST-DATE','GEN-DIST-DUR']
  layerIndex = layers.index(layer)
  startday = datetime.datetime.strptime('20211001','%Y%m%d')
  with open('mapTimeSeries_'+layer+'_'+ANNname+'.csv','w') as outts:
    outts.write("ID,maxLabel,Long,Lat")
    for day in range(0,365):
      outts.write(","+(startday + datetime.timedelta(days = day)).strftime("%Y%m%d"))
    outts.write("\n")
    for ID in range(1,301):
      (ID,Stratum,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS) = sampleFull[str(ID)].split(',')
      with open(outfolder + '/'+ID+'_DIST-ALERT_'+ANNname+'.csv','r') as map:
        grans = map.readlines()[1:]
      dates = ["20210930"]
      labels = [nodatavalue]
      maxlabel = -1000
      daily = []
      for g in grans:
        fields = g.strip().split(',') #(gID,sensingTime,ProductionTime,VEG_DIST_STATUS,VEG_ANOM,VEG_IND,VEG_DIST_CONF,VEG_DIST_DATE,	VEG_DIST_DUR,GEN_DIST_STATUS,GEN_ANOM,GEN_DIST_CONF,GEN_DIST_DATE,GEN_DIST_DUR)
        value = fields[layerIndex]
        if value == "":
          value = "NA"
        dates.append(fields[1])
        labels.append(value)
        if value != "NA" and int(value) != int(nodatavalue) and int(value) != -2 and int(value) != 200 and int(value) > maxlabel:
          maxlabel = int(value)
      dates.append("20221001")
      labels.append(nodatavalue)
      i = 0
      for day in range(0,365):
        if datetime.datetime.strptime(dates[i],'%Y%m%d') < startday + datetime.timedelta(days = day):
          while datetime.datetime.strptime(dates[i],'%Y%m%d') < startday + datetime.timedelta(days = day):
            i+=1        
        if datetime.datetime.strptime(dates[i],'%Y%m%d') == startday + datetime.timedelta(days = day):
          daily.append(labels[i])
          i += 1
        elif datetime.datetime.strptime(dates[i],'%Y%m%d') > startday + datetime.timedelta(days = day):
          daily.append(nodatavalue)         
      outts.write(','.join([ID,str(maxlabel),Long,Lat]))
      for d in daily:
        outts.write(","+d)
      outts.write("\n")

In [5]:
def makeRefTimeSeries(suffix):
  with open('reference'+suffix+'.csv','r') as file:
    lines = file.readlines()
  startday = datetime.datetime.strptime('20211001','%Y%m%d')
  with open('referenceTimeSeries'+suffix+'.csv','w') as outts:
    outts.write("ID,overallLabel,Long,Lat,changetype")
    for day in range(0,365):
      outts.write(","+(startday + datetime.timedelta(days = day)).strftime("%Y%m%d"))
    outts.write("\n")
    for ID in range(1,301):
      fields = lines[ID].strip().split(',')
      (ID,person,overallLabel,Long,Lat,UTMzone,centxUTM,centyUTM,changetype) = fields[0:9]
      fields = fields[9:]
      dates = ["20210930"]
      labels = ["noObs"]
      daily = []
      for period in range(0,73):
        dates.append(fields[period*2])
        labels.append(fields[period*2+1])
      dates.append("20221001")
      labels.append("noObs")
      i = 0
      for day in range(0,365):
        while dates[i] == "NA":
          i+=1
        if datetime.datetime.strptime(dates[i],'%Y%m%d') == startday + datetime.timedelta(days = day):
          daily.append(labels[i])
          i += 1
        elif datetime.datetime.strptime(dates[i],'%Y%m%d') > startday + datetime.timedelta(days = day):
          if labels[i-1] == labels[i]:
            daily.append(labels[i])
          else:
            daily.append("noObs")
        elif datetime.datetime.strptime(dates[i],'%Y%m%d') < startday + datetime.timedelta(days = day):
          if labels[i] == labels[i+1]:
            daily.append(labels[i])
          else:
            daily.append("noObs")
          i+=1
          
      outts.write(','.join([ID,overallLabel,Long,Lat,changetype]))
      for d in daily:
        outts.write(","+d)
      outts.write("\n")

In [8]:
#makeRefTimeSeries('_last')
makeMapTimeSeriesDaily('VEG-DIST-STATUS',255)
#makeMapTimeSeriesDaily('GEN-DIST-STATUS',255)